# Catalan Numbers

Looking at coding interview questions, I noticed there's an interesting set of problems that can be reduced to the computation of [Catalan numbers](https://en.wikipedia.org/wiki/Catalan_number). In this notebook, we'll look at some of these problems and I'll show you how to solve them. I expect this to be a ~20 minute read.


What is are Catalan numbers? Mathematically, we can find the $n$th Catalan number $C_n$ using the following closed form formula:



## $C_n = \frac{1}{n + 1} \binom{2n}{n}$

The first 10 catalan numbers, (starting from $n=0$):

1, 1, 2, 5, 14, 42, 132, 429, 1430, 4862...

To me, the formula above doesn't look very intuitive right away. Let's look at a couple of examples to see where these problems come up, and the formula will start to make more sense.

## Count Unique Binary Search Trees

Given $n$ values, how many unique binary trees can be constructed with $n$ nodes?
Here are some examples:

<img src="./images/catalan_trees.png">

One way to think about this problem is using recursion to compute all possible combinations of left and right subtrees:

- Fix one node as the root.
- Allocate $k$ nodes in the left subtree, and $n - k - 1$ nodes in the right subtree.
- Count the number of unique left subtrees of size $k$ nodes.
- Count the number of unique right subtrees of size $n - k - 1$ nodes.
- The number of unique structures for this allocation is the product of these two counts.
- Repeat this for $k=0,1,...,n-1$
- Sum the totals for all possible allocations.

Or, we can look at this recursive formula:
$B(n)=\sum_{k=0}^{n-1}B(k) B(n-k-1)$

where $B(n)$ is the number of unique binary trees with $n$ nodes.

Here's the code:

In [1]:
def count_trees(n):
    # Base cases
    if n == 0 or n == 1:
        return 1
    
    # Keep track of the total number of trees we can make
    total = 0
    
    # For each allocation of k nodes in the left subtree
    for k in range(n):
        # Count the number of trees in the left and right subtrees
        # Note: this can be improved with memoization.
        left_counts = count_trees(k)
        right_counts = count_trees(n - k - 1)
        
        # Add to the total the total possible structures.
        total += right_counts * left_counts
        
    return total

# Let's print some of the values
for n in range(6):
    print(f'count_trees({n}): {count_trees(n)}')

count_trees(0): 1
count_trees(1): 1
count_trees(2): 2
count_trees(3): 5
count_trees(4): 14
count_trees(5): 42


As we can see, the solution for input $n$ is the $n$th Catalan number! In fact, it would make perfect sense if I just renamed the `count_trees` function above to `catalan`.

I started with this example because it'll help us think about the next problem.

## Generate Parenthesis
Given a number $n$, generate all sequences of well-formed pairs of $n$ parentheses. Here are some examples:

```
n = 1:

()

n = 2:

(()) ()()

n = 3:

((())) (()()) (())() ()(()) ()()()
```


When we first look at this, we're probably thinking recursion and backtracking right? Let's code the recursive solution:

In [2]:
# Recursion/Backtracking solution

def generate_parentheses(n):
    # Initialize solution and current to empty lists
    solution = []
    current = []
    
    # Call recursive helper function with n left and n right parentheses
    helper(n, n, current, solution)
    
    return solution
    
def helper(left, right, current, solution):
    # If there are no more parentheses to use, update solution with current parentheses sequence
    if left == 0 and right == 0:
        solution.append(''.join(current))
        return
    
    # If there are still left parentheses to be used, we can add one and recurse
    if left > 0:
        current.append('(')
        helper(left - 1, right, current, solution)
        current.pop()  # backtrack

    # We can only add a right parentheses if we've already placed as many left parentheses
    if right > left:
        current.append(')')
        helper(left, right - 1, current, solution)
        current.pop()  # backtrack
    

for n in range(5):
    print(f"generate_parentheses({n}): {generate_parentheses(n)}")

generate_parentheses(0): ['']
generate_parentheses(1): ['()']
generate_parentheses(2): ['(())', '()()']
generate_parentheses(3): ['((()))', '(()())', '(())()', '()(())', '()()()']
generate_parentheses(4): ['(((())))', '((()()))', '((())())', '((()))()', '(()(()))', '(()()())', '(()())()', '(())(())', '(())()()', '()((()))', '()(()())', '()(())()', '()()(())', '()()()()']


Great! This is a good solultion. The runtime is ~$O(2^n)$ as we're constructing a tree of height $n$ with each level $i$ being the combinations for $i$ pairs of parentheses.

We also see that the number valid sequences for $n$ parentheses is equal to the $n$th Catalan number. How can we relate this problem to the problem above? In the first problem, we were counting the unique binary tree structures. In this problem, <b>we can represent each sequence of parentheses with a unique binary tree structure.</b>

Consider the solution for $n=1$: `()`. 

When I increase to $n=2$ pairs, I want to find somewhere to add `()` to the solution for $n=1$. I can either add them around the previous solution: `(())`, or I can add them to the right: `()()`.

In general, we can get all valid sequences of $n$ pairs of parentheses by finding all combinations of the form `(left_sequence) right_sequence`, where `left_sequence` is a valid sequence of parentheses containing $k$ pairs and `right_sequence` is a valid sequence of parentheses containing $n - k - 1$ pairs. 

```
n = 2
-----
    k = 1: (())
    k = 0: ()() 
    
n = 3
-----
    k = 2: ((())), (()())
    k = 1: (())()
    k = 0: ()(()), ()()()
```

To formulate this `(left_sequence) right_sequence` as a tree structure, we can treat the `left_sequence` as a left subtree, the `right_sequence` as a right subtree, and the parentheses around the left sequence as the root.

<img src="./images/parentheses.png">

In [3]:
# Alternate solution

def generate_parentheses(n):
    # subtree[i] contains a list of the possible subsequences with i pairs
    subtree = [['']]
    
    # We'll enumerate all of the possible sequences from 1 to n pairs
    for i in range(1, n + 1):
        # To construct trees of size i...
        
        # Start with an empty solution set
        subtree.append([])

        # Go through possible values of k
        for k in range(i):
            for left in subtree[k]:
                for right in subtree[i - k - 1]:
                    # Append a solution for each left subtree of size k and right subtree of size i - k - 1
                    subtree[i].append('(' + left + ')' + right)
    
    # Return the solution set for n pairs of parentheses
    return subtree[n]

for n in range(5):
    print(f"generate_parentheses({n}): {generate_parentheses(n)}")

generate_parentheses(0): ['']
generate_parentheses(1): ['()']
generate_parentheses(2): ['()()', '(())']
generate_parentheses(3): ['()()()', '()(())', '(())()', '(()())', '((()))']
generate_parentheses(4): ['()()()()', '()()(())', '()(())()', '()(()())', '()((()))', '(())()()', '(())(())', '(()())()', '((()))()', '(()()())', '(()(()))', '((())())', '((()()))', '(((())))']


Cool! Note that the runtime is still going to be exponential, and there's not a lot we can do about it. Catalan numbers grow exponentially with $n$ as we notice from our binomial coefficient term.

Now, I want to look at one more question which will solidify our understanding of Catalan numbers.

## Restricted Grid Paths
Given an $n \times n$ grid, find the number of ways to get from the top left corner to the bottom right corner, given that we can only move <b>down</b> or <b>right</b>.

<img src='./images/grid.png'>
<i>Get from point A to point B.</i>

You may have heard this question before. One way to think of this question is to think of the moves we make as a string of R's and D's to represent right moves and down moves. In total, to move from the top left corner to the bottom right corner, we would need to make $n$ <b>rights</b> and $n$ <b>downs</b>, in any order.
For example, we can enumerate the paths for $n=3$:

```
RRRDDD
RRDRDD
RDRRDD
DRRRDD
DRRDRD
...
```
The total number of combinations we can make will be $\binom{2n}{n}$ (out of $2n$ moves, choose $n$ to be D's. The rest are R's)

Here's a modified version of this question: Find the number of paths from the top left corner to the bottom right corner <b>under the constraint that no path may go below the diagonal</b>.

Here's an example for $n=3$. Note that <b>we cannot move below the dotted red diagonal line</b>, and all paths are within the upper right triangle of the grid:

<img src='./images/restricted_grid.png'>

We need to make $n$ <b>right</b> moves and $n$ <b>down</b> moves, but under this new constraint, we can never make more <b>down</b> moves than we have already made of <b>right</b> moves.

This sounds a lot like the parentheses question, where we could needed to place $n$ `(`'s and $n$ `)`'s, under the constraint that we can place no more `)`'s than we've already placed of `(`'s. In fact, if we treat R's as `(`'s and D's as `)`'s, it becomes the exact same problem.

```
Solution for n = 3:
RRRDDD, RRDRDD, RRDDRD, RDRRDD, RDRDRD
((())), (()()), (())(), ()(()), ()()()
```

Therefore, once again, the solution to this problem can be reduced to the calculation of Catalan numbers. Here's a function to explicitly compute the Catalan numbers:

In [4]:
import scipy.special

# Using scipy.special.binom for the binomial coefficient
def catalan(n):
    return int(1 / (n + 1) * scipy.special.binom(2 * n, n))

## Combinatorics Intuition

Notice that without the constraint, the solution for a grid of size $n$ is $\binom{2n}{n}$. With the constraint, the solution becomes the $n$th Catalan number, $C_n = \frac{1}{n + 1} \binom{2n}{n}$. 

What exactly is this constraint? Well, we can express $C_n$ as:

$C_n = \frac{1}{n+1} \binom{2n}{n} = \binom{2n}{n} - \binom{2n}{n + 1}$.

This follows because we can show that $\binom{2n}{n + 1} = \frac{n}{n+1} \binom{2n}{n}$:

$\binom{2n}{n + 1}$

$= \frac{2n}{(2n - (n + 1))! (n + 1)!}$

$= \frac{2n}{(2n - (n + 1))! (n + 1) n!}$

$= \frac{n}{n+1}\frac{2n}{(2n - n)! n!}$

$= \frac{n}{n+1} \binom{2n}{n}$

Therefore, of the $\binom{2n}{n}$ solutions to the unconstrained problem, there are $\binom{2n}{n + 1}$ solutions that violate the constraint ("bad solutions"). 

One way to see this is by thinking about the Grid Paths problem again. We want to count the number of "bad solutions". Of our $2n$ moves, instead of choosing $n$ <b>down</b> moves, we can choose $n + 1$ moves to be $n$ <b>down</b> moves and $1$ <b>right</b> move. By choosing $n + 1$ moves to manipulate, we can get each of these combination violates the constraint, as we can arrange the moves such that $n$ <b>down</b> moves have appeared before the $n$th <b>right</b> move*.

Thus, we get $\binom{2n}{n + 1}$ invalid solutions. Note that since $2n$ is even, $\binom{2n}{n + 1}$ is equal to $\binom{2n}{n - 1}$. (We can alternatively think of choosing $n - 1$ right moves.)  

Below, the purple represents the $n + 1$ moves we choose to violate the constraint for $n = 2$.

<img src='./images/intuition.png'>

<i>*In this explanation, there are other conditions that I've made assumptions on (particularly that a unique solution can be found for each of these combinations). This assumption is justified by the reflection proof on Wikipedia!</i>


There are other better, more rigorous, and more interesting intuitions on [Wikipedia](https://en.wikipedia.org/wiki/Catalan_number). I won't reproduce them here, but many of them are quite easy to follow through, so I encourage you to look at them if you're interested!